In [2]:
#Parameters
dt=0.01

T=3
Tank_he=0.02
Caisson_he=0.02
Scales=[21, 25]

#Visualization
Triangles=True
Air=False



import sys
import proteus
## Required imports
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so, Comm, Context
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread
#import Context

import tempFlume as plant
reload(plant)
import tempFlume_so as plant_so
reload(plant_so)

import pandas as pd
import ipympl
from ipywidgets import *
import matplotlib.pyplot as plt
import utilities
reload(utilities)
import helpers
reload(helpers)
from IPython.display import clear_output

f1 = plt.figure(num=1)
plt.plot([])
display(VBox([f1.canvas]))
for s_ind, scale in enumerate(Scales):
    #scale=25
    print s_ind, scale 
    xl = pd.ExcelFile("DoE_TempFlume.xlsx")
    df1 = xl.parse(0)
    df2 = xl.parse(1)
    df3 = xl.parse(2)
    df4 = xl.parse(3)
    df5 = xl.parse(4)
    #print [key for key in dict(df.dtypes)]
    for index in range(df4.shape[0]):
        clear_output()
        reload(Context)
        reload(plant)
        reload(plant_so)
        print('Simulating Experiment #',index)

        xl = pd.ExcelFile("DoE_TempFlume.xlsx")
        df1 = xl.parse(0)
        df2 = xl.parse(1)
        df3 = xl.parse(2)
        df4 = xl.parse(3, header=0, index_col=0)
        df5 = xl.parse(4, header=0, index_col=0)

        if scale==21:
            df_DoE=df4
            df_data=df2

        elif scale==25:
            df_DoE=df5
            df_data=df3
        else:
            print 'Wrong Scale'
            () + 1
        status=df_DoE['Finished'].tolist()[index]
        #if status == 'Completed':
        #    continue
        iFlow=df_DoE['Flow Speed'].tolist()[index]
        iDepth=df_DoE['Depth'].tolist()[index]
        iOffset=df_DoE['Offset'].tolist()[index]
        tagFlow=df_data['Speed Tags'].tolist()[iFlow]
        tagDepth=df_data['Depth Tags'].tolist()[iDepth]
        tagOffset=df_data['Offset Tags'].tolist()[iOffset]

        flowSpeed=df_data['Flow Speed'].tolist()[iFlow]
        waterLevel=df_data['Depth'].tolist()[iDepth]
        offset=df_data['Offset'].tolist()[iOffset]
        print('Speed Real={0}, Depth Real={1}, Offset Weight={2}, Scale={3}'.format(tagFlow,
                                                                             tagDepth,
                                                                             tagOffset,
                                                                             scale)
              
         # Fixed Parameters
        caissonDim=[8.6,1.3]  #Approximate Dimension of original caisson

        #Change Context
        plant.opts.water_level=waterLevel
        plant.opts.inflow_velocity=flowSpeed
        plant.opts.outflow_velocity=flowSpeed
        plant.opts.caisson_scale=float(scale)
        plant.opts.caisson_Yoffset=-offset


        plant.opts.dt_fixed = dt
        plant.opts.T = T
        plant.opts.he = Tank_he
        plant.opts.he_caisson=Caisson_he

        plant.Update_Model()

        Context.setFromModule(plant,mutable=True)
        ct = Context.get()
        helpers.f=plt.figure(num=1)
        helpers.plot_domain()
        # numerical simulation options
        plant_so.ct=ct
        so = plant_so
        so.tnList = [0.0,plant.dt_init]+[i*plant.dt_fixed for i in range(1,plant.nDTout+1)]
        info = open("TimeList.txt","w")
        for time in so.tnList:
            info.write(str(time)+"\n")
        info.close()
        plant_so.tnList=so.tnList
        pList=[]
        nList=[]
        so.sList=[]
        OptDB = PETSc.Options()
        for (p,n) in so.pnList:
            so.sList.append(default_s)
            pList.append(__import__(p))
            reload(pList[-1])
            nList.append(__import__(n))
            reload(nList[-1])
            pList[-1].name = p
            nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
            nList[-1].levelLinearSolver = default_n.KSP_petsc4py
            OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
            OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
            OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
        opts.save_dof = True
        opts.dataDir='.'
        opts.probDir='.'
        opts.logLevel=7
        opts.verbose=True
        opts.viewMesh=True

        #-----------------------------------------------------------------------
        ## Numerical Solution Object
        #    Now we create an the numerical wavetank object and set it up to run in a thread on each engine.
        #-----------------------------------------------------------------------

        print("Creating Numerical Solution Object...")
        #import time
        #t1=time.time()
        ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)
        # Create a thread wrapper for the simulation.  The target must be an argument-less
        # function so we wrap the call to `calculateSolution` in a simple lambda:
        print("Creating Simulation Thread...")
        simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
        print("Starting Thread and Simulation Monitor")
        simulation_thread.start()

        utilities.simulation_thread=simulation_thread
        utilities.ns=ns
        utilities.plt=plt
        utilities.f=plt.figure(num=1)
        utilities.plant=plant
        utilities.showTriangles=Triangles
        utilities.showAir=Air
        error=utilities.monitor_simulation(refresh=5.0)
        import os   
        checkpath=os.path.exists('./Results') 
        if not checkpath:
                os.mkdir("Results")
        ExpTag='T_' + str(index) + "_S_"+ str(scale)
        if not error:
            !rm ./Results/{ExpTag}_forceHistory_p.txt
            !rm ./Results/{ExpTag}_forceHistory_v.txt
            !cp forceHistory_p.txt ./Results/{ExpTag}_forceHistory_p.txt
            !cp forceHistory_v.txt  ./Results/{ExpTag}_forceHistory_v.txt
            Exp_Status='Completed'
        else:
            Exp_Status='Crashed'
        #!rm ./Results/{ExpTag}_p.xmf
        #!rm ./Results/{ExpTag}_p.h5
        #!cp tempFlume_p.xmf ./Results/{ExpTag}_p.xmf
        #!cp tempFlume_p.h5  ./Results/{ExpTag}_p.h5
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('DoE_TempFlume.xlsx', engine='xlsxwriter')

        if scale==21:
            df4.loc[index,'Finished']=Exp_Status

        elif scale==25:
            df5.loc[index,'Finished']=Exp_Status
        else:
            print 'Scale Error'

        # Convert the dataframe to an XlsxWriter Excel object.
        df1.to_excel(writer, sheet_name='Parameters')
        df2.to_excel(writer, sheet_name='Parameters Scale 1-21')
        df3.to_excel(writer, sheet_name='Parameters Scale 1-25')
        df4.to_excel(writer, sheet_name='Tests Scale 1-21')
        df5.to_excel(writer, sheet_name='Tests Scale 1-25')
        # Close the Pandas Excel writer and output the Excel file.
        writer.save()

('Simulating Experiment #', 22)
Speed Real=10, Depth Real=5.0, Offset Weight=0.5 lbs, Scale=25
Creating Numerical Solution Object...
Creating Simulation Thread...
Starting Thread and Simulation Monitor
u_max=0.000, v_max=0.000, Vmax=0.000, cfl=0.000, dt=1.00000
Monitored for: 0:00:05.056688. at t= 1.00000e-04
u_max=0.690, v_max=0.562, Vmax=0.690, cfl=0.012, dt=0.00020
Monitored for: 0:00:15.865309. at t= 7.00000e-04
u_max=0.746, v_max=0.671, Vmax=0.746, cfl=0.094, dt=0.00160
Monitored for: 0:00:29.113553. at t= 1.50000e-02
u_max=0.811, v_max=0.716, Vmax=0.811, cfl=0.294, dt=0.00500
Monitored for: 0:00:43.739352. at t= 4.00000e-02
u_max=0.784, v_max=0.699, Vmax=0.784, cfl=58.573, dt=0.01000
Monitored for: 0:00:57.776800. at t= 7.00000e-02
u_max=0.775, v_max=0.695, Vmax=0.775, cfl=0.845, dt=0.01442
Monitored for: 0:01:13.008025. at t= 1.00000e-01
u_max=0.767, v_max=0.730, Vmax=0.774, cfl=0.598, dt=0.01000
Monitored for: 0:01:30.093052. at t= 1.30000e-01
u_max=0.762, v_max=0.587, Vmax=0.7

Monitored for: 0:16:57.285013. at t= 1.95000e+00
u_max=0.834, v_max=0.561, Vmax=0.834, cfl=0.625, dt=0.01000
Monitored for: 0:17:11.867073. at t= 1.98000e+00
u_max=0.831, v_max=0.562, Vmax=0.831, cfl=62.627, dt=0.01000
Monitored for: 0:17:26.856165. at t= 2.01000e+00
u_max=0.831, v_max=0.581, Vmax=0.831, cfl=0.628, dt=0.01000
Monitored for: 0:17:41.262684. at t= 2.03000e+00
u_max=0.785, v_max=0.593, Vmax=0.785, cfl=64.447, dt=0.01000
Monitored for: 0:17:52.090218. at t= 2.05000e+00
u_max=0.859, v_max=0.572, Vmax=0.859, cfl=0.650, dt=0.01000
Monitored for: 0:18:05.309683. at t= 2.08000e+00
u_max=0.796, v_max=0.576, Vmax=0.796, cfl=0.667, dt=0.01000
Monitored for: 0:18:15.822412. at t= 2.10000e+00
u_max=3.443, v_max=1.846, Vmax=3.443, cfl=0.651, dt=0.01000
Monitored for: 0:18:27.508343. at t= 2.11606e+00
u_max=0.733, v_max=0.534, Vmax=0.733, cfl=0.309, dt=0.00500
Monitored for: 0:18:41.609454. at t= 2.14000e+00
u_max=0.731, v_max=0.502, Vmax=0.731, cfl=0.612, dt=0.01000
Monitored for: 0:

OSError: [Errno 12] Cannot allocate memory

In [ ]:
#! pip install pandas
#! pip install xlsxwriter
#! pip install ipywidgets
#! pip install ipympl
#! jupyter nbextension enable --py --sys-prefix ipympl